# 🏗️ Module 8.1 - Architecture de Production NLP

## Objectifs d'apprentissage
- Concevoir une architecture microservices pour NLP
- Implémenter une API FastAPI robuste
- Configurer Redis pour le cache et les queues
- Intégrer monitoring et health checks
- Déployer avec Docker et docker-compose

## Vue d'ensemble
Ce notebook vous guide dans la création d'une architecture de production complète pour services NLP, capable de gérer des milliers de requêtes par seconde avec haute disponibilité.

## 🎯 Architecture Cible

```
┌─────────────┐    ┌──────────────┐    ┌─────────────────┐
│   Client    │───▶│ Load Balancer│───▶│   API Gateway   │
└─────────────┘    └──────────────┘    └─────────────────┘
                                                │
                                                ▼
                                    ┌─────────────────────┐
                                    │  Service Discovery  │
                                    └─────────────────────┘
                                                │
                   ┌────────────────────────────┼────────────────────────────┐
                   ▼                            ▼                            ▼
          ┌─────────────────┐        ┌─────────────────┐        ┌─────────────────┐
          │ Sentiment Service│        │    NER Service  │        │   QA Service    │
          └─────────────────┘        └─────────────────┘        └─────────────────┘
                   │                            │                            │
                   └────────────────────────────┼────────────────────────────┘
                                                ▼
                                    ┌─────────────────────┐
                                    │    Redis Cache      │
                                    │  & Message Queue    │
                                    └─────────────────────┘
                                                │
                                                ▼
                                    ┌─────────────────────┐
                                    │   PostgreSQL DB     │
                                    └─────────────────────┘
```

## 📦 Configuration de l'environnement

In [ ]:
# Installation des dépendances nécessaires
!pip install fastapi uvicorn redis prometheus-client psycopg2-binary
!pip install transformers tensorflow
!pip install celery flower
!pip install python-multipart jinja2

In [ ]:
import os
import asyncio
import logging
import json
import time
from datetime import datetime
from typing import Dict, List, Optional, Any
import hashlib

# FastAPI et middlewares
from fastapi import FastAPI, HTTPException, Depends, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field

# Cache et monitoring
import redis
from prometheus_client import Counter, Histogram, Gauge, generate_latest

# NLP
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import tensorflow as tf

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 🔧 Configuration et Classes de Base

In [ ]:
class Config:
    """Configuration centralisée pour l'application"""
    
    # API Configuration
    API_HOST = os.getenv("API_HOST", "0.0.0.0")
    API_PORT = int(os.getenv("API_PORT", "8000"))
    API_WORKERS = int(os.getenv("API_WORKERS", "4"))
    
    # Redis Configuration
    REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
    REDIS_PORT = int(os.getenv("REDIS_PORT", "6379"))
    REDIS_DB = int(os.getenv("REDIS_DB", "0"))
    REDIS_PASSWORD = os.getenv("REDIS_PASSWORD")
    
    # Cache TTL (Time To Live) en secondes
    CACHE_TTL_SHORT = 300    # 5 minutes
    CACHE_TTL_MEDIUM = 3600  # 1 heure
    CACHE_TTL_LONG = 86400   # 24 heures
    
    # Model Configuration
    MODEL_CACHE_DIR = os.getenv("MODEL_CACHE_DIR", "./models")
    DEFAULT_MODEL = "bert-base-uncased"
    
    # Rate Limiting
    RATE_LIMIT_PER_MINUTE = int(os.getenv("RATE_LIMIT_PER_MINUTE", "100"))
    
    # Health Check
    HEALTH_CHECK_INTERVAL = 30
    
    # Monitoring
    ENABLE_METRICS = os.getenv("ENABLE_METRICS", "true").lower() == "true"
    METRICS_PORT = int(os.getenv("METRICS_PORT", "9090"))

config = Config()
print("✅ Configuration chargée")
print(f"Redis: {config.REDIS_HOST}:{config.REDIS_PORT}")
print(f"API: {config.API_HOST}:{config.API_PORT}")

In [ ]:
# Modèles Pydantic pour validation des données
class TextInput(BaseModel):
    text: str = Field(..., min_length=1, max_length=5000, description="Texte à analyser")
    model: Optional[str] = Field(default="bert-base", description="Modèle à utiliser")
    language: Optional[str] = Field(default="en", description="Langue du texte")
    
class SentimentResponse(BaseModel):
    text: str
    sentiment: str
    confidence: float
    processing_time: float
    model_used: str
    cached: bool = False
    
class HealthStatus(BaseModel):
    status: str
    timestamp: datetime
    version: str
    uptime_seconds: float
    services: Dict[str, str]
    
class BatchTextInput(BaseModel):
    texts: List[str] = Field(..., max_items=100, description="Liste de textes à analyser")
    model: Optional[str] = Field(default="bert-base", description="Modèle à utiliser")
    
print("✅ Modèles de données définis")

## 🧠 Service Manager pour NLP

In [ ]:
class NLPModelManager:
    """Gestionnaire des modèles NLP avec cache et optimisations"""
    
    def __init__(self):
        self.models = {}
        self.model_stats = {}
        self.startup_time = time.time()
        
    def load_model(self, model_name: str, task: str = "sentiment-analysis"):
        """Charge un modèle en mémoire avec gestion d'erreurs"""
        try:
            if model_name not in self.models:
                logger.info(f"Chargement du modèle {model_name} pour {task}")
                
                # Configuration optimisée selon la tâche
                if task == "sentiment-analysis":
                    model = pipeline(
                        task,
                        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                        framework="tf",
                        device=0 if tf.config.list_physical_devices('GPU') else -1
                    )
                else:
                    model = pipeline(task, model=model_name, framework="tf")
                
                self.models[model_name] = model
                self.model_stats[model_name] = {
                    "loaded_at": datetime.now(),
                    "predictions": 0,
                    "total_time": 0.0
                }
                
                logger.info(f"✅ Modèle {model_name} chargé avec succès")
                
            return self.models[model_name]
            
        except Exception as e:
            logger.error(f"❌ Erreur lors du chargement de {model_name}: {e}")
            raise HTTPException(status_code=500, detail=f"Erreur modèle: {e}")
    
    def predict_sentiment(self, text: str, model_name: str = "sentiment") -> Dict:
        """Prédiction de sentiment avec métriques"""
        start_time = time.time()
        
        try:
            model = self.load_model(model_name, "sentiment-analysis")
            
            # Prédiction
            result = model(text)
            
            # Traitement du résultat
            if isinstance(result, list) and len(result) > 0:
                pred = result[0]
                sentiment = pred['label'].lower()
                confidence = pred['score']
            else:
                sentiment = "neutral"
                confidence = 0.5
                
            processing_time = time.time() - start_time
            
            # Mise à jour des statistiques
            self.model_stats[model_name]["predictions"] += 1
            self.model_stats[model_name]["total_time"] += processing_time
            
            return {
                "sentiment": sentiment,
                "confidence": confidence,
                "processing_time": processing_time,
                "model_used": model_name
            }
            
        except Exception as e:
            logger.error(f"Erreur prédiction: {e}")
            raise HTTPException(status_code=500, detail="Erreur de prédiction")
    
    def get_model_stats(self) -> Dict:
        """Retourne les statistiques des modèles"""
        stats = {}
        for model_name, model_stat in self.model_stats.items():
            avg_time = (
                model_stat["total_time"] / model_stat["predictions"]
                if model_stat["predictions"] > 0 else 0
            )
            
            stats[model_name] = {
                "predictions": model_stat["predictions"],
                "average_time": round(avg_time, 4),
                "total_time": round(model_stat["total_time"], 2),
                "loaded_since": str(model_stat["loaded_at"])
            }
            
        return stats

# Instance globale du gestionnaire de modèles
nlp_manager = NLPModelManager()
print("✅ NLP Model Manager initialisé")

## 🔄 Service de Cache Redis

In [ ]:
class RedisCache:
    """Service de cache Redis avec gestion de connexion robuste"""
    
    def __init__(self):
        self.client = None
        self.connected = False
        self.connect()
        
    def connect(self):
        """Établit la connexion Redis avec retry automatique"""
        try:
            self.client = redis.Redis(
                host=config.REDIS_HOST,
                port=config.REDIS_PORT,
                db=config.REDIS_DB,
                password=config.REDIS_PASSWORD,
                decode_responses=True,
                socket_timeout=5,
                socket_connect_timeout=5,
                retry_on_timeout=True
            )
            
            # Test de connexion
            self.client.ping()
            self.connected = True
            logger.info("✅ Redis connecté")
            
        except Exception as e:
            logger.warning(f"⚠️ Redis non disponible: {e}")
            self.connected = False
            self.client = None
    
    def generate_key(self, text: str, model: str, additional_data: Dict = None) -> str:
        """Génère une clé de cache déterministe"""
        # Normalisation du texte
        normalized_text = text.strip().lower()
        
        # Données pour le hash
        key_data = {
            "text": normalized_text,
            "model": model
        }
        
        if additional_data:
            key_data.update(additional_data)
        
        # Hash MD5 pour une clé courte et unique
        key_string = json.dumps(key_data, sort_keys=True)
        key_hash = hashlib.md5(key_string.encode()).hexdigest()
        
        return f"nlp:sentiment:{key_hash}"
    
    async def get(self, key: str) -> Optional[Dict]:
        """Récupère une valeur du cache"""
        if not self.connected:
            return None
            
        try:
            value = self.client.get(key)
            if value:
                return json.loads(value)
        except Exception as e:
            logger.error(f"Erreur lecture cache: {e}")
            
        return None
    
    async def set(self, key: str, value: Dict, ttl: int = None) -> bool:
        """Stocke une valeur dans le cache"""
        if not self.connected:
            return False
            
        try:
            ttl = ttl or config.CACHE_TTL_MEDIUM
            self.client.setex(key, ttl, json.dumps(value))
            return True
        except Exception as e:
            logger.error(f"Erreur écriture cache: {e}")
            return False
    
    async def delete(self, key: str) -> bool:
        """Supprime une clé du cache"""
        if not self.connected:
            return False
            
        try:
            return bool(self.client.delete(key))
        except Exception as e:
            logger.error(f"Erreur suppression cache: {e}")
            return False
    
    def get_stats(self) -> Dict:
        """Retourne les statistiques Redis"""
        if not self.connected:
            return {"status": "disconnected"}
            
        try:
            info = self.client.info()
            return {
                "status": "connected",
                "used_memory": info.get("used_memory_human", "N/A"),
                "connected_clients": info.get("connected_clients", 0),
                "total_commands_processed": info.get("total_commands_processed", 0),
                "keyspace_hits": info.get("keyspace_hits", 0),
                "keyspace_misses": info.get("keyspace_misses", 0)
            }
        except Exception as e:
            logger.error(f"Erreur stats Redis: {e}")
            return {"status": "error", "error": str(e)}

# Instance globale du cache
cache = RedisCache()
print(f"✅ Cache Redis initialisé (connecté: {cache.connected})")

## 📊 Système de Métriques Prometheus

In [ ]:
# Métriques Prometheus pour monitoring
if config.ENABLE_METRICS:
    # Compteurs
    REQUEST_COUNTER = Counter(
        'nlp_requests_total',
        'Nombre total de requêtes NLP',
        ['method', 'endpoint', 'status', 'model']
    )
    
    # Histogrammes pour la latence
    REQUEST_LATENCY = Histogram(
        'nlp_request_duration_seconds',
        'Durée des requêtes NLP en secondes',
        ['method', 'endpoint', 'model'],
        buckets=[0.01, 0.05, 0.1, 0.25, 0.5, 1.0, 2.5, 5.0, 10.0]
    )
    
    # Histogramme pour la confiance des prédictions
    CONFIDENCE_HISTOGRAM = Histogram(
        'nlp_prediction_confidence',
        'Distribution des scores de confiance',
        ['model', 'sentiment'],
        buckets=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    )
    
    # Gauges pour l'état actuel
    CACHE_HIT_RATE = Gauge(
        'nlp_cache_hit_rate',
        'Taux de succès du cache Redis'
    )
    
    ACTIVE_MODELS = Gauge(
        'nlp_active_models',
        'Nombre de modèles chargés en mémoire'
    )
    
    MEMORY_USAGE = Gauge(
        'nlp_memory_usage_bytes',
        'Utilisation mémoire des modèles',
        ['model']
    )

class MetricsCollector:
    """Collecteur de métriques avec calculs avancés"""
    
    def __init__(self):
        self.request_count = 0
        self.cache_hits = 0
        self.cache_total = 0
        
    def record_request(self, method: str, endpoint: str, status: int, 
                      model: str, duration: float, cached: bool = False):
        """Enregistre une requête avec toutes ses métriques"""
        if not config.ENABLE_METRICS:
            return
            
        self.request_count += 1
        
        # Compteur de requêtes
        REQUEST_COUNTER.labels(
            method=method,
            endpoint=endpoint,
            status=str(status),
            model=model
        ).inc()
        
        # Latence
        REQUEST_LATENCY.labels(
            method=method,
            endpoint=endpoint,
            model=model
        ).observe(duration)
        
        # Cache hit rate
        self.cache_total += 1
        if cached:
            self.cache_hits += 1
            
        if self.cache_total > 0:
            hit_rate = self.cache_hits / self.cache_total
            CACHE_HIT_RATE.set(hit_rate)
    
    def record_prediction(self, model: str, sentiment: str, confidence: float):
        """Enregistre les métriques d'une prédiction"""
        if not config.ENABLE_METRICS:
            return
            
        CONFIDENCE_HISTOGRAM.labels(
            model=model,
            sentiment=sentiment
        ).observe(confidence)
    
    def update_model_metrics(self):
        """Met à jour les métriques des modèles"""
        if not config.ENABLE_METRICS:
            return
            
        ACTIVE_MODELS.set(len(nlp_manager.models))
        
        # Simulation de l'utilisation mémoire (en production, utiliser psutil)
        for model_name in nlp_manager.models.keys():
            # Estimation approximative pour BERT
            estimated_memory = 500 * 1024 * 1024  # 500MB
            MEMORY_USAGE.labels(model=model_name).set(estimated_memory)

# Instance globale du collecteur
metrics = MetricsCollector()
print("✅ Système de métriques Prometheus configuré")

## 🚀 Application FastAPI

In [ ]:
# Création de l'application FastAPI
app = FastAPI(
    title="NLP Production API",
    description="API de production pour services NLP avec cache, monitoring et haute disponibilité",
    version="2.1.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Configuration CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # En production, spécifier les domaines autorisés
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Variables globales pour le monitoring
app_start_time = time.time()

In [ ]:
# Middleware pour métriques automatiques
@app.middleware("http")
async def metrics_middleware(request, call_next):
    """Middleware pour collecter automatiquement les métriques"""
    start_time = time.time()
    
    # Traitement de la requête
    response = await call_next(request)
    
    # Calcul des métriques
    process_time = time.time() - start_time
    
    # Extraction des informations
    method = request.method
    path = request.url.path
    status_code = response.status_code
    
    # Modèle utilisé (depuis les headers de réponse si disponible)
    model_used = response.headers.get("X-Model-Used", "unknown")
    
    # Enregistrement des métriques
    metrics.record_request(
        method=method,
        endpoint=path,
        status=status_code,
        model=model_used,
        duration=process_time,
        cached=response.headers.get("X-Cache-Hit", "false") == "true"
    )
    
    # Ajout des headers de performance
    response.headers["X-Process-Time"] = str(round(process_time, 4))
    
    return response

print("✅ Middleware de métriques configuré")

## 🔗 Endpoints de l'API

In [ ]:
@app.get("/health", response_model=HealthStatus)
async def health_check():
    """Health check complet avec statut des services"""
    uptime = time.time() - app_start_time
    
    # Vérification des services
    services_status = {
        "redis": "healthy" if cache.connected else "unhealthy",
        "nlp_models": "healthy" if len(nlp_manager.models) > 0 else "warming_up",
        "api": "healthy"
    }
    
    # Statut global
    overall_status = "healthy" if all(
        status in ["healthy", "warming_up"] for status in services_status.values()
    ) else "degraded"
    
    return HealthStatus(
        status=overall_status,
        timestamp=datetime.now(),
        version="2.1.0",
        uptime_seconds=uptime,
        services=services_status
    )

@app.get("/ready")
async def readiness_check():
    """Readiness probe pour Kubernetes"""
    # Vérification que les services critiques sont prêts
    if not cache.connected:
        raise HTTPException(status_code=503, detail="Redis non disponible")
    
    return {"status": "ready", "timestamp": datetime.now()}

@app.get("/metrics")
async def get_metrics():
    """Endpoint pour Prometheus scraping"""
    if not config.ENABLE_METRICS:
        raise HTTPException(status_code=404, detail="Métriques désactivées")
    
    # Mise à jour des métriques avant export
    metrics.update_model_metrics()
    
    return Response(
        content=generate_latest(),
        media_type="text/plain"
    )

print("✅ Endpoints de santé configurés")

In [ ]:
@app.post("/sentiment", response_model=SentimentResponse)
async def analyze_sentiment(input_data: TextInput):
    """Analyse de sentiment avec cache intelligent"""
    start_time = time.time()
    
    # Génération de la clé de cache
    cache_key = cache.generate_key(
        input_data.text, 
        input_data.model,
        {"language": input_data.language}
    )
    
    # Tentative de récupération du cache
    cached_result = await cache.get(cache_key)
    if cached_result:
        cached_result["cached"] = True
        
        # Headers pour le monitoring
        response = JSONResponse(content=cached_result)
        response.headers["X-Cache-Hit"] = "true"
        response.headers["X-Model-Used"] = cached_result["model_used"]
        
        return response
    
    # Prédiction avec le modèle
    try:
        result = nlp_manager.predict_sentiment(input_data.text, input_data.model)
        
        # Enrichissement du résultat
        result["text"] = input_data.text
        result["cached"] = False
        
        # Mise en cache du résultat
        await cache.set(cache_key, result, config.CACHE_TTL_MEDIUM)
        
        # Enregistrement des métriques de prédiction
        metrics.record_prediction(
            model=result["model_used"],
            sentiment=result["sentiment"],
            confidence=result["confidence"]
        )
        
        # Headers pour le monitoring
        response = JSONResponse(content=result)
        response.headers["X-Cache-Hit"] = "false"
        response.headers["X-Model-Used"] = result["model_used"]
        
        return response
        
    except Exception as e:
        logger.error(f"Erreur analyse sentiment: {e}")
        raise HTTPException(status_code=500, detail="Erreur lors de l'analyse")

@app.post("/sentiment/batch")
async def analyze_sentiment_batch(input_data: BatchTextInput):
    """Analyse de sentiment en lot pour traitement optimisé"""
    results = []
    
    for text in input_data.texts:
        try:
            # Réutilisation de la logique individuelle
            text_input = TextInput(text=text, model=input_data.model)
            result = await analyze_sentiment(text_input)
            
            # Extraction du contenu JSON si c'est une Response
            if hasattr(result, 'body'):
                import json
                result = json.loads(result.body)
            
            results.append(result)
            
        except Exception as e:
            logger.error(f"Erreur sur le texte '{text[:50]}...': {e}")
            results.append({
                "text": text,
                "error": str(e),
                "sentiment": "error",
                "confidence": 0.0
            })
    
    return {
        "total_processed": len(input_data.texts),
        "successful": len([r for r in results if "error" not in r]),
        "results": results
    }

print("✅ Endpoints d'analyse de sentiment configurés")

In [ ]:
@app.get("/stats")
async def get_service_stats():
    """Statistiques détaillées des services"""
    uptime = time.time() - app_start_time
    
    return {
        "service": {
            "name": "NLP Production API",
            "version": "2.1.0",
            "uptime_seconds": uptime,
            "uptime_human": f"{uptime//3600:.0f}h {(uptime%3600)//60:.0f}m {uptime%60:.0f}s"
        },
        "models": nlp_manager.get_model_stats(),
        "cache": cache.get_stats(),
        "requests": {
            "total": metrics.request_count,
            "cache_hit_rate": round(metrics.cache_hits / max(metrics.cache_total, 1), 3)
        }
    }

@app.get("/models")
async def list_available_models():
    """Liste des modèles disponibles et leurs statuts"""
    available_models = {
        "sentiment": {
            "name": "Sentiment Analysis",
            "description": "Analyse de sentiment (positif/négatif/neutre)",
            "languages": ["en", "fr"],
            "loaded": "sentiment" in nlp_manager.models,
            "model_id": "cardiffnlp/twitter-roberta-base-sentiment-latest"
        }
    }
    
    return {
        "available_models": available_models,
        "loaded_models": list(nlp_manager.models.keys()),
        "total_loaded": len(nlp_manager.models)
    }

@app.delete("/cache")
async def clear_cache():
    """Vide le cache Redis (utile pour le debugging)"""
    if not cache.connected:
        raise HTTPException(status_code=503, detail="Redis non disponible")
    
    try:
        cache.client.flushdb()
        return {"message": "Cache vidé avec succès", "timestamp": datetime.now()}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erreur: {e}")

print("✅ Endpoints d'administration configurés")

## 🧪 Tests et Validation

In [ ]:
# Tests de base pour valider l'architecture
import asyncio
from fastapi.testclient import TestClient

# Client de test
client = TestClient(app)

def test_health_endpoint():
    """Test de l'endpoint de santé"""
    response = client.get("/health")
    assert response.status_code == 200
    
    data = response.json()
    assert "status" in data
    assert "services" in data
    assert "uptime_seconds" in data
    
    print("✅ Test health endpoint : OK")
    return data

def test_sentiment_analysis():
    """Test de l'analyse de sentiment"""
    test_data = {
        "text": "I love this product! It's amazing!",
        "model": "sentiment",
        "language": "en"
    }
    
    response = client.post("/sentiment", json=test_data)
    assert response.status_code == 200
    
    data = response.json()
    assert "sentiment" in data
    assert "confidence" in data
    assert "processing_time" in data
    
    print(f"✅ Test sentiment analysis : {data['sentiment']} (confiance: {data['confidence']:.3f})")
    return data

def test_cache_functionality():
    """Test du système de cache"""
    test_text = "This is a test for caching functionality."
    
    # Première requête (cache miss)
    response1 = client.post("/sentiment", json={"text": test_text})
    data1 = response1.json()
    
    # Deuxième requête (cache hit attendu)
    response2 = client.post("/sentiment", json={"text": test_text})
    data2 = response2.json()
    
    # Vérification que le cache fonctionne
    cache_hit = response2.headers.get("X-Cache-Hit") == "true"
    
    print(f"✅ Test cache : Cache hit = {cache_hit}")
    print(f"   Première requête: {data1['processing_time']:.4f}s")
    print(f"   Deuxième requête: {data2['processing_time']:.4f}s")
    
    return cache_hit

# Exécution des tests
print("🧪 Lancement des tests...\n")

health_data = test_health_endpoint()
sentiment_data = test_sentiment_analysis()
cache_working = test_cache_functionality()

print("\n✅ Tous les tests sont passés !")
print(f"Service status: {health_data['status']}")
print(f"Cache Redis: {'✅' if cache.connected else '❌'}")
print(f"Modèles chargés: {len(nlp_manager.models)}")

## 📋 Test de Charge et Performance

In [ ]:
import concurrent.futures
import statistics
import threading
from collections import defaultdict

def performance_test(num_requests=50, concurrent_threads=10):
    """Test de performance avec requêtes concurrentes"""
    
    test_texts = [
        "I love this product!",
        "This is terrible quality.",
        "The service was okay, nothing special.",
        "Amazing experience, highly recommended!",
        "Could be better, but acceptable.",
        "Worst purchase ever, very disappointed.",
        "Good value for money.",
        "Excellent customer service and fast delivery.",
        "Not what I expected, returning it.",
        "Perfect! Exactly what I wanted."
    ]
    
    results = []
    errors = []
    
    def make_request(text):
        """Fonction pour une requête individuelle"""
        try:
            start_time = time.time()
            response = client.post("/sentiment", json={"text": text})
            end_time = time.time()
            
            if response.status_code == 200:
                data = response.json()
                return {
                    "success": True,
                    "duration": end_time - start_time,
                    "cached": data.get("cached", False),
                    "confidence": data.get("confidence", 0)
                }
            else:
                return {"success": False, "status_code": response.status_code}
                
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    print(f"🚀 Lancement test de charge: {num_requests} requêtes avec {concurrent_threads} threads")
    
    # Génération des requêtes
    requests_to_make = []
    for i in range(num_requests):
        text = test_texts[i % len(test_texts)]
        requests_to_make.append(text)
    
    # Exécution concurrente
    start_time = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_threads) as executor:
        future_to_text = {executor.submit(make_request, text): text for text in requests_to_make}
        
        for future in concurrent.futures.as_completed(future_to_text):
            result = future.result()
            if result["success"]:
                results.append(result)
            else:
                errors.append(result)
    
    total_time = time.time() - start_time
    
    # Analyse des résultats
    if results:
        durations = [r["duration"] for r in results]
        cached_requests = sum(1 for r in results if r["cached"])
        
        stats = {
            "total_requests": num_requests,
            "successful_requests": len(results),
            "failed_requests": len(errors),
            "total_time": round(total_time, 2),
            "requests_per_second": round(len(results) / total_time, 2),
            "avg_latency": round(statistics.mean(durations) * 1000, 2),  # en ms
            "p50_latency": round(statistics.median(durations) * 1000, 2),
            "p95_latency": round(statistics.quantiles(durations, n=20)[18] * 1000, 2),
            "p99_latency": round(statistics.quantiles(durations, n=100)[98] * 1000, 2),
            "min_latency": round(min(durations) * 1000, 2),
            "max_latency": round(max(durations) * 1000, 2),
            "cache_hit_rate": round(cached_requests / len(results) * 100, 1),
            "success_rate": round(len(results) / num_requests * 100, 1)
        }
        
        print("\n📊 Résultats du test de performance:")
        print(f"   Requêtes totales: {stats['total_requests']}")
        print(f"   Succès: {stats['successful_requests']} ({stats['success_rate']}%)")
        print(f"   Échecs: {stats['failed_requests']}")
        print(f"   Durée totale: {stats['total_time']}s")
        print(f"   Req/sec: {stats['requests_per_second']}")
        print(f"   Latence moyenne: {stats['avg_latency']}ms")
        print(f"   Latence P95: {stats['p95_latency']}ms")
        print(f"   Latence P99: {stats['p99_latency']}ms")
        print(f"   Cache hit rate: {stats['cache_hit_rate']}%")
        
        return stats
    else:
        print("❌ Aucune requête réussie")
        return None

# Exécution du test de performance
perf_stats = performance_test(num_requests=100, concurrent_threads=20)

if perf_stats:
    print("\n🎯 Évaluation des performances:")
    if perf_stats['requests_per_second'] > 50:
        print("   ✅ Excellent throughput")
    elif perf_stats['requests_per_second'] > 20:
        print("   ⚠️ Throughput acceptable")
    else:
        print("   ❌ Throughput faible")
        
    if perf_stats['p95_latency'] < 100:
        print("   ✅ Excellente latence P95")
    elif perf_stats['p95_latency'] < 500:
        print("   ⚠️ Latence P95 acceptable")
    else:
        print("   ❌ Latence P95 élevée")

## 🐳 Configuration Docker

In [ ]:
# Création des fichiers de configuration Docker

dockerfile_content = '''# Dockerfile pour API NLP de production
FROM python:3.9-slim

# Variables d'environnement
ENV PYTHONUNBUFFERED=1
ENV PYTHONDONTWRITEBYTECODE=1

# Création utilisateur non-root
RUN useradd --create-home --shell /bin/bash nlpuser

# Dépendances système
RUN apt-get update && apt-get install -y \
    build-essential \
    && rm -rf /var/lib/apt/lists/*

# Répertoire de travail
WORKDIR /app

# Installation des dépendances Python
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copie de l'application
COPY . .
RUN chown -R nlpuser:nlpuser /app

# Pré-téléchargement des modèles
RUN python -c "from transformers import pipeline; pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest')"

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

# Utilisateur non-root
USER nlpuser

# Port exposé
EXPOSE 8000

# Commande de démarrage
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "4"]
'''

print("📝 Dockerfile généré")
print(dockerfile_content[:500] + "...")

In [ ]:
# docker-compose.yml pour environnement complet
docker_compose_content = '''version: '3.8'

services:
  # API NLP
  nlp-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - REDIS_HOST=redis
      - REDIS_PORT=6379
      - ENABLE_METRICS=true
    depends_on:
      - redis
      - postgres
    volumes:
      - ./models:/app/models
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3

  # Cache Redis
  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"
    volumes:
      - redis_data:/data
    command: redis-server --appendonly yes
    restart: unless-stopped

  # Base de données PostgreSQL
  postgres:
    image: postgres:15-alpine
    environment:
      - POSTGRES_DB=nlp_production
      - POSTGRES_USER=nlpuser
      - POSTGRES_PASSWORD=nlppassword123
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data
    restart: unless-stopped

  # Monitoring Prometheus
  prometheus:
    image: prom/prometheus:latest
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
      - prometheus_data:/prometheus
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'
      - '--storage.tsdb.path=/prometheus'
      - '--web.console.libraries=/etc/prometheus/console_libraries'
      - '--web.console.templates=/etc/prometheus/consoles'
    restart: unless-stopped

  # Dashboard Grafana
  grafana:
    image: grafana/grafana:latest
    ports:
      - "3000:3000"
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin123
    volumes:
      - grafana_data:/var/lib/grafana
    restart: unless-stopped

  # Load Balancer Nginx
  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
    depends_on:
      - nlp-api
    restart: unless-stopped

volumes:
  redis_data:
  postgres_data:
  prometheus_data:
  grafana_data:

networks:
  default:
    name: nlp-production
'''

print("📝 docker-compose.yml généré")
print("Services inclus: API NLP, Redis, PostgreSQL, Prometheus, Grafana, Nginx")

In [ ]:
# Configuration Nginx pour load balancing
nginx_config = '''events {
    worker_connections 1024;
}

http {
    upstream nlp_backend {
        least_conn;
        server nlp-api:8000 weight=1 max_fails=3 fail_timeout=30s;
        # Ajouter plus d'instances ici pour scaling horizontal
    }

    # Cache configuration
    proxy_cache_path /var/cache/nginx levels=1:2 keys_zone=nlp_cache:10m max_size=1g
                     inactive=60m use_temp_path=off;

    server {
        listen 80;
        
        # Rate limiting
        limit_req_zone $binary_remote_addr zone=api_limit:10m rate=10r/s;
        
        location / {
            proxy_pass http://nlp_backend;
            
            # Headers
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
            
            # Timeouts
            proxy_connect_timeout 30s;
            proxy_send_timeout 30s;
            proxy_read_timeout 30s;
            
            # Rate limiting
            limit_req zone=api_limit burst=20 nodelay;
        }
        
        # Cache pour les requêtes GET non-critiques
        location ~ ^/(health|models|stats) {
            proxy_pass http://nlp_backend;
            proxy_cache nlp_cache;
            proxy_cache_valid 200 5m;
            proxy_cache_use_stale error timeout updating http_500 http_502 http_503 http_504;
            add_header X-Cache-Status $upstream_cache_status;
        }
        
        # Health check endpoint pour load balancer
        location /nginx-health {
            access_log off;
            return 200 "healthy\n";
            add_header Content-Type text/plain;
        }
    }
}
'''

print("📝 Configuration Nginx générée")
print("Features: Load balancing, rate limiting, caching, health checks")

## 📊 Dashboard de Monitoring

In [ ]:
# Visualisation simple des métriques
import matplotlib.pyplot as plt
import numpy as np

def create_monitoring_dashboard():
    """Crée un dashboard simple de monitoring"""
    
    # Données simulées pour la démonstration
    hours = list(range(24))
    
    # Simulation de trafic avec pics aux heures de bureau
    traffic = [50 + 30*np.sin((h-6)*np.pi/12) + np.random.normal(0, 10) for h in hours]
    traffic = [max(0, t) for t in traffic]  # Pas de valeurs négatives
    
    # Simulation de latence (plus élevée pendant les pics)
    latency = [30 + 20*np.sin((h-6)*np.pi/12) + np.random.normal(0, 5) for h in hours]
    latency = [max(10, l) for l in latency]  # Latence minimale de 10ms
    
    # Cache hit rate (plus élevé pendant les heures de forte charge)
    cache_hit_rate = [60 + 20*np.sin((h-6)*np.pi/12) + np.random.normal(0, 5) for h in hours]
    cache_hit_rate = [max(20, min(95, c)) for c in cache_hit_rate]  # Entre 20% et 95%
    
    # Création du dashboard
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('📊 Dashboard NLP Production API - Dernières 24h', fontsize=16, fontweight='bold')
    
    # Graphique 1: Trafic
    ax1.plot(hours, traffic, 'b-', linewidth=2, marker='o', markersize=4)
    ax1.set_title('🚀 Trafic (Requêtes/heure)', fontweight='bold')
    ax1.set_xlabel('Heure')
    ax1.set_ylabel('Requêtes/heure')
    ax1.grid(True, alpha=0.3)
    ax1.fill_between(hours, traffic, alpha=0.3, color='blue')
    
    # Graphique 2: Latence
    ax2.plot(hours, latency, 'r-', linewidth=2, marker='s', markersize=4)
    ax2.set_title('⚡ Latence P95 (ms)', fontweight='bold')
    ax2.set_xlabel('Heure')
    ax2.set_ylabel('Latence (ms)')
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=100, color='orange', linestyle='--', alpha=0.7, label='Seuil alerte')
    ax2.legend()
    
    # Graphique 3: Cache Hit Rate
    ax3.plot(hours, cache_hit_rate, 'g-', linewidth=2, marker='^', markersize=4)
    ax3.set_title('💾 Cache Hit Rate (%)', fontweight='bold')
    ax3.set_xlabel('Heure')
    ax3.set_ylabel('Cache Hit Rate (%)')
    ax3.grid(True, alpha=0.3)
    ax3.fill_between(hours, cache_hit_rate, alpha=0.3, color='green')
    ax3.set_ylim(0, 100)
    
    # Graphique 4: Distribution des sentiments
    sentiments = ['Positif', 'Neutre', 'Négatif']
    sentiment_counts = [45, 35, 20]  # Pourcentages simulés
    colors = ['#4CAF50', '#FFC107', '#F44336']
    
    ax4.pie(sentiment_counts, labels=sentiments, colors=colors, autopct='%1.1f%%', startangle=90)
    ax4.set_title('😊 Distribution des Sentiments', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('nlp_dashboard.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Résumé des métriques
    print("\n📈 Résumé des métriques (24h):")
    print(f"   Trafic total: {sum(traffic):.0f} requêtes")
    print(f"   Latence moyenne: {np.mean(latency):.1f}ms")
    print(f"   Cache hit rate moyen: {np.mean(cache_hit_rate):.1f}%")
    print(f"   Pic de trafic: {max(traffic):.0f} req/h à {hours[traffic.index(max(traffic))]}h")
    
    return {
        'total_requests': sum(traffic),
        'avg_latency': np.mean(latency),
        'avg_cache_hit_rate': np.mean(cache_hit_rate),
        'peak_hour': hours[traffic.index(max(traffic))],
        'peak_traffic': max(traffic)
    }

# Génération du dashboard
dashboard_stats = create_monitoring_dashboard()

## 🎯 Conclusion et Recommandations

### ✅ Ce que nous avons implémenté :

1. **Architecture Microservices**
   - API FastAPI avec validation Pydantic
   - Gestionnaire de modèles NLP optimisé
   - Cache Redis intelligent avec TTL adaptatif
   - Système de métriques Prometheus

2. **Fonctionnalités de Production**
   - Health checks et readiness probes
   - Rate limiting et gestion d'erreurs
   - Middleware de métriques automatique
   - Support du traitement par lots

3. **Monitoring et Observabilité**
   - Métriques RED (Rate, Errors, Duration)
   - Métriques business (confiance, distribution sentiment)
   - Dashboard de performance
   - Logs structurés

4. **Déploiement**
   - Configuration Docker optimisée
   - docker-compose pour environnement complet
   - Load balancing Nginx
   - Scaling horizontal ready

### 🚀 Étapes suivantes pour la production :

1. **Sécurité**
   - Authentification JWT
   - Rate limiting par utilisateur
   - Validation et sanitisation des inputs
   - HTTPS avec certificats SSL

2. **Scaling**
   - Kubernetes pour orchestration
   - Auto-scaling basé sur métriques
   - Load balancing multi-région
   - CDN pour cache global

3. **Monitoring Avancé**
   - Alerting Prometheus/Grafana
   - Distributed tracing (Jaeger)
   - Log aggregation (ELK)
   - ML monitoring (drift detection)

4. **Optimisations**
   - Model quantization
   - GPU utilization
   - Connection pooling
   - Async processing

### 📊 Métriques de Succès :
- **Latence P95** : < 100ms
- **Throughput** : > 1000 req/sec
- **Disponibilité** : > 99.9%
- **Cache Hit Rate** : > 70%
- **Error Rate** : < 0.1%

Cette architecture est prête pour supporter des charges de production réelles et peut être facilement étendue selon les besoins spécifiques de votre organisation.